# Определение возраста покупателей

"Хлеб-Соль" - супермаркет, который планирует внедрить систему компьютерного зрения для того, чтобы обеспечить возможность определения возраста покупателей при помощи их фотофиксации в прикассовой зоне. Предполагается, что подобная система позволит более эффективно проводить рекомендации товаров на основе возраста покупателя. Кроме этого, это позволит контролировать добросовестность кассиров при продаже алкоголя.

**Целью данного проекта** будет являться построение модели машинного обучения, которая позволит на основе входных данных о покупателе, а именно фотографии, определять возраст этого человека. С точки зрения машинного обучения будем решать задачу регрессии по предсказанию количества лет. 

В ходе проекта мы проведем исследовательский анализ данных, обучение модели и изучение качества построенной модели. В качестве исходной, мы воспользуемся архитектурой нейронной сети **ResNet50**,  доказавшей свое успешное применение для подобного рода задач. 

In [2]:
#Импортируем все библиотеки, используемые в проекте

#import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential 
import numpy as np 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet import ResNet50
import pandas as pd
import matplotlib.pyplot as plt

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## 1. Исследовательский анализ данных 

Проведем исследовательский анализ данных. Для этого предварительно изучим размер выборки, исследуем график распределения возраста в выборке, выведем первые несколько фотографий на и посмотрим как устроен предлагаемый датасет.

Определим **размер** представленной выборки. 

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')

train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 
features, target = train_gen_flow.next()

print(features.shape)

**Вывод**: в исходном датасете найдено 7591 цветных изображения, которые мы свели к размеру (224,224). 

Изучим теперь **распределение по возрастам** исходного датасета.  

In [ ]:
plt.hist(labels['real_age'], bins = 32)
plt.title('Распределение возраста в выборке')
plt.xlabel('Возраст')
plt.ylabel('Количество');

**Вывод**: из полученного графика видно, что распределение по возрастам не равномерное. Большая часть людей имеет возраст в промежутке от 20 до 40. Имеется ярко выделенный пик в районе 27 лет, максимальный возраст не превышает 100 лет. После 40 лет наблюдается плавное уменьшение количества людей в соответсвующих возрастных категориях. 

Изучим основые статистические характеристики.

In [ ]:
labels['real_age'].describe()

**Вывод**: средний возраст участников составляет 31 год, максимальный возраст 100 лет, минимальный 1 год.

Для получения представления об имеющейся выборке выведем 12 изображений из входного датасета. 

In [ ]:
plt.figure(figsize=(14, 14))
for i in range (12):
    ax = plt.subplot(4, 3, i+1)
    plt.imshow(features[i])
    plt.title(target[i], weight ='bold' )
    


**Вывод**: можно отметить, что на некоторых фотоснимках по краям кадра представлен черный фон, который, очевидно, не вносит информации о возрасте изображенного человека. 

**Общий вывод**: исходный датасет представляет из себя набор цветных фотографий с изображением лиц людей в количестве 7591 и размером (224, 224). Распределение возрастов людей в представленной выборке неравномерное, средней возраст составляет 31 год, максимальный возраст 100 лет, минимальный возраст 1 год. 

## 2. Обучение модели 

Для обучения модели реализуем набор функций - загрузки тренировочной выборки, загрузки тестовой выборки, создания модели, и обучения модели. 

Для загрузки тренировочных и тестовых выборок выспользуемся динамической выгрузкой данных при помощи ImageDataGenerator. Тренировочную и тестовую выборки возьмем в соотношении 1:3. Проведем также допольнительную аугументацию для расширения представленной выборки данных.

In [ ]:
#Загрузка тренировочных данных 
def load_train(path): 
    labels = pd.read_csv(path+'labels.csv')
    train_datagen = ImageDataGenerator(
        validation_split = 0.25, 
        rescale = 1/255., 
        horizontal_flip = True, 
        vertical_flip = False,
        width_shift_range = 0.2, 
        height_shift_range = 0.2
    )
    
    train_dataflow = train_datagen.flow_from_dataframe(
        dataframe=labels, 
        directory = path + 'final_files/',
        x_col = 'file_name',
        y_col = 'real_age', 
        class_mode = 'raw', 
        subset = 'training', 
        target_size = (150, 150),
        batch_size = 16,
        seed=12345
    )
    return train_dataflow

In [ ]:
#Загрузка тестовых данных
def load_test(path): 
   
     labels = pd.read_csv(path + 'labels.csv')
     validation_datagen = ImageDataGenerator(validation_split = .25, rescale = 1/255.)
     val_dataflow = validation_datagen.flow_from_dataframe(
         dataframe = labels, 
         directory = path + 'final_files/',
         x_col = 'file_name',
         y_col = 'real_age', 
         class_mode = 'raw', 
         subset = 'validation', 
         target_size = (150, 150), 
         batch_size = 16,
         seed=12345

     )
     return val_dataflow

Наша модель будет представлять из себя нейронную сеть ResNet50, для обучения которой применим функцию потерь MSE. Метрикой качества будет выступать MAE. Поскольку перед нами стоит задача регрессии, то в последнем слое Dense необходимо использовать один нейрон с функцией активации relu. 

In [ ]:
def create_model(input_shape): 

    backbone = ResNet50(input_shape=input_shape,
                 include_top=False,
                 weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5') 
    optimizer = Adam(lr=0.00001) 
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu')) 
    model.compile(optimizer=optimizer, loss='mse', 
                  metrics=['mae']) 
    return model 

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=5, 
               steps_per_epoch=None, validation_steps=None): 
 
    model.fit(train_data, 
              validation_data=test_data, 
              batch_size=batch_size, epochs=epochs, 
              steps_per_epoch=steps_per_epoch, 
              validation_steps=validation_steps, 
              verbose=2, shuffle=True) 
    return model

## 3. Изучение качества модели.

Вывод после обучения модели:

```python
    
Логи
``` 
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2023-11-30 06:11:38.708141: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-11-30 06:11:38.742699: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2099995000 Hz
2023-11-30 06:11:38.744768: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x58e0cb0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-30 06:11:38.744802: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-30 06:11:38.952918: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4f40fd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-30 06:11:38.952954: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-11-30 06:11:38.954652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-11-30 06:11:38.954707: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-11-30 06:11:38.954717: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-11-30 06:11:38.954745: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-11-30 06:11:38.954754: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-11-30 06:11:38.954763: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-11-30 06:11:38.954772: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-11-30 06:11:38.954779: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-11-30 06:11:38.957675: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-11-30 06:11:38.959620: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-11-30 06:11:40.729147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-11-30 06:11:40.729195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2023-11-30 06:11:40.729203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2023-11-30 06:11:40.732381: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-11-30 06:11:40.732437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10240 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 356 steps, validate for 119 steps
Epoch 1/5
2023-11-30 06:11:57.872178: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-11-30 06:11:59.074873: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 91s - loss: 695.3710 - mae: 21.3760 - val_loss: 1096.7945 - val_mae: 28.5859
Epoch 2/5
356/356 - 62s - loss: 178.8472 - mae: 10.0967 - val_loss: 256.5995 - val_mae: 11.7413
Epoch 3/5
356/356 - 61s - loss: 117.5086 - mae: 8.2208 - val_loss: 109.0846 - val_mae: 7.7807
Epoch 4/5
356/356 - 62s - loss: 100.6645 - mae: 7.7173 - val_loss: 96.5059 - val_mae: 7.4247
Epoch 5/5
356/356 - 61s - loss: 90.4780 - mae: 7.2512 - val_loss: 96.4591 - val_mae: 7.4822
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
119/119 - 9s - loss: 96.4591 - mae: 7.4822
**Test MAE: 7.4822**

В результате тренировки модели и проверки качества на тестовой выборке были получены следующие результаты: 

119/119 - 9s - loss: 96.4591 - mae: 7.4822 <br>
**Test MAE: 7.4822**

## Выводы 

Результатом работы в ходе данного проекта  является разработка системы компьютерного зрения для определения возраста покупателей в прикассовой зоне супермаркета “Хлеб-Соль”. Это позволит компании более эффективно рекомендовать товары на основе возраста клиента и контролировать продажу алкоголя. Для решения задачи использовалась модель машинного обучения на основе архитектуры нейронной сети ResNet50. В рамках проекта будет проведен исследовательский анализ данных, обучение модели и оценка ее качества.

В ходе работы была построена модель нейронной сети ResNet50 (включающую в себя 50 слоев) и с предварительно подготовлеными весами модели. Для подбора величины шага градиентного спуска был использован алгоритм Adam с параметром **learning_rate = 0.00001**. Для обучения модели использовалось 5 эпох.  

В результате проверки качества построенной модели на тестовой выборке был получен результат по метрике **MAE = 7.48**. 

Теоретически можно увеличить наш результат, изменив шаг обучения в алгоритме Адам и увеличив количество эпох.